<a href="https://colab.research.google.com/github/Ignas12345/Magistro_projektas/blob/main/knygutes/TCGA_isoformu_atsisiuntimas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
BiocManager::install("TCGAbiolinks", ask = FALSE, verbose = TRUE)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.20 (BiocManager 1.30.25), R 4.4.2 (2024-10-31)

Installing package(s) 'BiocVersion', 'TCGAbiolinks'

system (cmd0): /usr/lib/R/bin/R CMD INSTALL

also installing the dependencies ‘plogr’, ‘png’, ‘Biostrings’, ‘RSQLite’, ‘KEGGREST’, ‘filelock’, ‘UCSC.utils’, ‘GenomeInfoDbData’, ‘zlibbioc’, ‘matrixStats’, ‘abind’, ‘SparseArray’, ‘AnnotationDbi’, ‘BiocFileCache’, ‘BiocGenerics’, ‘GenomeInfoDb’, ‘XVector’, ‘R.oo’, ‘R.methodsS3’, ‘MatrixGenerics’, ‘Biobase’, ‘S4Arrays’, ‘DelayedArray’, ‘downloader’, ‘biomaRt’, ‘GenomicRanges’, ‘XML’, ‘plyr’, ‘IRanges’, ‘S4Vectors’, ‘R.utils’, ‘SummarizedExperiment’, ‘TCGAbiolinksGUI.data’


foundpkgs: plogr, png, Biostrings, RSQLite, KEGGREST, filelock, UCSC.utils, GenomeInfoDbData, zlibbioc, matrixStats, abind, SparseArray, AnnotationDbi, BiocF

In [51]:
library(TCGAbiolinks)

In [5]:
clinical_data <- GDCquery_clinic(project = "TCGA-TGCT", type = "clinical")

In [6]:
teratoma_submitter_ids <- clinical_data$submitter_id[clinical_data$primary_diagnosis == "Teratoma, benign"]
teratoma_submitter_ids <- teratoma_submitter_ids[!is.na(teratoma_submitter_ids)]

# View the result
teratoma_submitter_ids

[1] "TCGA-2G-AAG5" "TCGA-2G-AAG6" "TCGA-2G-AAGY" "TCGA-2G-AAGV" "TCGA-2G-AAGX"

In [7]:
query_miRNA_1 <- GDCquery(
  project = "TCGA-TGCT",
  data.category = "Transcriptome Profiling",
  data.type = "Isoform Expression Quantification",
  barcode = teratoma_submitter_ids
)

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-TGCT

--------------------

oo Filtering results

--------------------

ooo By data.type

ooo By barcode

----------------

oo Checking data

----------------

ooo Checking if there are duplicated cases

ooo Checking if there are results for the query

-------------------

o Preparing output

-------------------



In [8]:
GDCdownload(query_miRNA_1)

# Prepare the data for analysis



GDCdownload will download 6 files. A total of 3.467809 MB




Downloading: 390 kB     

In [110]:
data_teratoma <- GDCprepare(query_miRNA_1)

  |======================================================================| 100%


In [ ]:
head(data_teratoma)

In [50]:
library(dplyr)
library(tidyr)

In [ ]:
# Combine miRNA_ID and miRNA_region into a single identifier
data_teratoma <- data_teratoma %>%
  mutate(miRNA_id_and_region = paste(miRNA_ID, miRNA_region, sep = ", "))

# This should be summing the read counts per million RNA mapped, but instead it sums the regular read counts
aggregated_data_teratoma <- data_teratoma %>%
  group_by(miRNA_id_and_region, barcode) %>%
  summarise(total_reads_per_million_miRNA_mapped = sum(reads_per_million_miRNA_mapped), .groups = "drop")

# Reshape into wide format
final_teratoma_reads_per_million_miRNA_mapped <- aggregated_data_teratoma %>%
  pivot_wider(names_from = barcode, values_from = total_reads_per_million_miRNA_mapped, values_fill = 0)

# View the resulting table
head(final_teratoma_reads_per_million_miRNA_mapped, n = 20)

In [ ]:
teratoma_reads <- as.data.frame(final_teratoma_reads_per_million_miRNA_mapped[,-1])
rownames(teratoma_reads) <- final_teratoma_reads_per_million_miRNA_mapped$miRNA_id_and_region

In [ ]:
head(teratoma_reads, n =10)


In [117]:
print(sum(teratoma_reads$'TCGA-2G-AAG5-01A-11R-A439-13'))

[1] 1e+06


In [ ]:
other_submitter_ids <- clinical_data$submitter_id[clinical_data$primary_diagnosis != "Teratoma, benign"]
other_submitter_ids <- other_submitter_ids[!is.na(other_submitter_ids)]

# View the result
other_submitter_ids

In [56]:
query_miRNA_2 <- GDCquery(
  project = "TCGA-TGCT",
  data.category = "Transcriptome Profiling",
  data.type = "Isoform Expression Quantification",
  barcode = other_submitter_ids
)

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-TGCT

--------------------

oo Filtering results

--------------------

ooo By data.type

ooo By barcode

----------------

oo Checking data

----------------

ooo Checking if there are duplicated cases

ooo Checking if there are results for the query

-------------------

o Preparing output

-------------------



In [57]:
GDCdownload(query_miRNA_2)



GDCdownload will download 133 files. A total of 72.845722 MB




Downloading: 8.1 MB     

In [113]:
data_other <- GDCprepare(query_miRNA_2)

  |======================================================================| 100%


In [ ]:
# Combine miRNA_ID and miRNA_region into a single identifier
data_other <- data_other %>%
  mutate(miRNA_id_and_region = paste(miRNA_ID, miRNA_region, sep = ", "))

# This should be summing the read counts per million RNA mapped, but instead it sums the regular read counts
aggregated_data_other <- data_other %>%
  group_by(miRNA_id_and_region, barcode) %>%
  summarise(total_reads_per_million_miRNA_mapped = sum(reads_per_million_miRNA_mapped), .groups = "drop")

# Reshape into wide format
final_other_reads_per_million_miRNA_mapped <- aggregated_data_other %>%
  pivot_wider(names_from = barcode, values_from = total_reads_per_million_miRNA_mapped, values_fill = 0)

# View the resulting table
head(final_other_reads_per_million_miRNA_mapped, n = 20)

In [88]:
other_reads <- as.data.frame(final_other_reads_per_million_miRNA_mapped[,-1])
rownames(other_reads) <- final_other_reads_per_million_miRNA_mapped$miRNA_id_and_region

In [ ]:
#teratoma_reads[grepl('371', rownames(teratoma_reads)),]

In [ ]:
#other_reads[grepl('371', rownames(other_reads)),]

In [121]:
filtered_teratoma = teratoma_reads[(rowSums(teratoma_reads[])>1*ncol(teratoma_reads)),]
print(nrow(filtered_teratoma))

[1] 801


In [122]:
filtered_other = other_reads[(rowSums(other_reads[])>1*ncol(other_reads)),]
print(nrow(filtered_other))

[1] 947


In [ ]:
#sujungiam naudodami chat GPT koda...

# Find the union of row names
all_rows <- union(rownames(filtered_teratoma), rownames(filtered_other))

# Reindex filtered_teratoma to include all rows, fill missing rows with 0
filtered_teratoma_full <- data.frame(matrix(0, nrow = length(all_rows), ncol = ncol(filtered_teratoma)))
rownames(filtered_teratoma_full) <- all_rows
colnames(filtered_teratoma_full) <- colnames(filtered_teratoma)
filtered_teratoma_full[rownames(filtered_teratoma), ] <- filtered_teratoma

# Reindex filtered_other to include all rows, fill missing rows with 0
filtered_other_full <- data.frame(matrix(0, nrow = length(all_rows), ncol = ncol(filtered_other)))
rownames(filtered_other_full) <- all_rows
colnames(filtered_other_full) <- colnames(filtered_other)
filtered_other_full[rownames(filtered_other), ] <- filtered_other

# Combine the two data frames by column
combined_df <- cbind(filtered_teratoma_full, filtered_other_full)

# View the resulting combined data frame
print(combined_df)


In [133]:
write.csv(combined_df, file = "filtered_isoform_read_per_million_RNA_mapped.csv", row.names = TRUE)